# Marketing Metrics

Using SQL to gather insights on the current state of the sales funnel. Turned into a weekly dashboard to present to the marketing team.

## **Opportunities**

Explore the opportunities table

**Opportunities that have Account Rank of "A"**

In [ ]:
/* Return all the opportunities where the related account has an account rank of "A" */
SELECT opportunity_type, opportunity_id, close_date
FROM Opportunities
WHERE account_id IN (SELECT account_id FROM Accounts WHERE account_rank = 'A')
ORDER BY opportunity_type

**Ranking Upsell Booking Opportunities by Projected ACV**

In [ ]:
/* Ranking Upsell Booking Opportunities by Projected ACV */
SELECT opportunity_id, projected_acv, 
       RANK() OVER (ORDER BY projected_acv DESC) AS projected_acv_rank
FROM Opportunities
WHERE opportunity_type LIKE 'Upsell Booking%';

## **Direct New Booking Pipeline**

**Total Direct New Booking Pipeline Coverage for Q3 2023** 

_Shows the sum of projected average contract value for all direct new booking opportunities that are still open and have a close date in Q3 2023_

In [ ]:
/* Direct New Booking Pipeline Coverage for Q3 2023 */
SELECT 
    SUM(projected_acv) as sum_acv
FROM Opportunities
WHERE 
    opportunity_type LIKE 'New Booking%' 
    AND customer_type = 'Direct' 
    AND stage <> 'Closed - Won' 
    AND stage <> 'Closed - Lost' 
    AND DATEPART(QUARTER, close_date) = 3 
    AND YEAR(close_date) = YEAR(CURRENT_TIMESTAMP)

**Direct New Booking Pipeline Coverage for Q3 2023 by Stage**

_Shows the sum of direct new booking pipeline by stage and the number of opportunities in each stage_

In [ ]:
/* Direct New Booking Pipeline Coverage for Q3 2023 by Stage */
SELECT 
    stage, 
    SUM(projected_acv) AS sum_acv,
    COUNT(*) AS num_records
FROM Opportunities
WHERE 
    opportunity_type LIKE 'New Booking%' 
    AND customer_type = 'Direct' 
    AND stage <> 'Closed - Won' 
    AND stage <> 'Closed - Lost' 
    AND DATEPART(QUARTER, close_date) = 3 
    AND YEAR(close_date) = YEAR(CURRENT_TIMESTAMP)
GROUP BY stage

**Direct New Booking Pipeline Coverage for Q3 2023 by Vertical**

_Shows the average ACV for Q3 2023 Pipeline Coverage by Vertical_

In [ ]:
/* Direct New Booking Pipeline for Q3 2023 by Vertical */
SELECT 
    ISNULL(a.vertical, 'Other') AS vertical, 
    ROUND(AVG(o.projected_acv),0) AS avg_projected_acv
FROM Opportunities o LEFT JOIN Accounts a ON o.account_id = a.account_id
WHERE 
    o.opportunity_type LIKE 'New Booking%' 
    AND o.customer_type = 'Direct' 
    AND o.stage <> 'Closed - Won' 
    AND o.stage <> 'Closed - Lost' 
    AND DATEPART(QUARTER, o.close_date) = 3 
    AND YEAR(o.close_date) = 2023
GROUP BY a.vertical
ORDER BY avg_projected_acv DESC

**Direct New Booking Pipeline Coverage for Q3 2023 by Expected Service Tier**

_Shows the average ACV for Q3 2023 Pipeline Coverage by EST_

In [ ]:
/* Avg ACV for Q3 2023 Pipeline Coverage by EST */
SELECT 
    ISNULL(a.expected_service_tier, 'Other') AS EST, 
    ROUND(AVG(o.projected_acv),0) AS avg_projected_acv
FROM Opportunities o LEFT JOIN Accounts a ON o.account_id = a.account_id
WHERE 
    o.opportunity_type LIKE 'New Booking%' 
    AND o.customer_type = 'Direct' 
    AND o.stage <> 'Closed - Won' 
    AND o.stage <> 'Closed - Lost' 
    AND DATEPART(QUARTER, o.close_date) = 3 
    AND YEAR(o.close_date) = 2023
GROUP BY a.expected_service_tier
ORDER BY a.expected_service_tier

**Total Direct New Booking Deals in Current Year**

_Shows the sum of the average contract value for each deal that was booked, the number of deals, and the account names associated with the deals in current year (2023) so far_

In [ ]:
/* shows the sum of ACV for each deal and the number of deals */
SELECT 
    COUNT(opportunity_id) AS num_deals, 
    ROUND(SUM(projected_acv),0) AS direct_new_bookings
FROM Opportunities
WHERE 
    opportunity_type LIKE 'New Booking%' 
    AND customer_type = 'Direct' 
    AND stage = 'Closed - Won' 
    AND YEAR(close_date) = YEAR(CURRENT_TIMESTAMP)

/* shows the account names for the opportunities that were closed in current year (2023) */
SELECT a.account_name, projected_acv AS ACV
FROM Opportunities o LEFT JOIN Accounts a ON o.account_id = a.account_id
WHERE 
    opportunity_type LIKE 'New Booking%' 
    AND customer_type = 'Direct' 
    AND stage = 'Closed - Won' 
    AND YEAR(close_date) = YEAR(CURRENT_TIMESTAMP)

**Total Pipeline Coverage for Future Quarters**

_Shows the sum of pipeline coverage for direct new booking opportunities for all future quarters (excluding the current quarter) and then splits it up by Vertical_

In [ ]:
/* total pipeline coverage for future quarters */
SELECT SUM(projected_acv)
FROM Opportunities
WHERE 
    opportunity_type LIKE 'New Booking%' 
    AND customer_type = 'Direct' 
    AND stage <> 'Closed - Won' 
    AND stage <> 'Closed - Lost' 
    AND (YEAR(close_date) <> YEAR(CURRENT_TIMESTAMP) OR DATEPART(QUARTER, close_date) <> DATEPART(QUARTER, CURRENT_TIMESTAMP))

/* count of opportunities, total pipeline coverage, and avg projected acv for future quarters by vertical */
SELECT 
    ISNULL(a.vertical, 'Other') AS vertical,
    COUNT(o.opportunity_id) AS num_opps,
    SUM(o.projected_acv) AS sum_proj_acv, 
    AVG(o.projected_acv) AS avg_proj_acv
FROM Opportunities o LEFT JOIN Accounts a ON o.account_id = a.account_id
WHERE 
    o.opportunity_type LIKE 'New Booking%' 
    AND o.customer_type = 'Direct' 
    AND o.stage <> 'Closed - Won' 
    AND o.stage <> 'Closed - Lost' 
    AND (YEAR(o.close_date) <> YEAR(CURRENT_TIMESTAMP) OR DATEPART(QUARTER, o.close_date) <> DATEPART(QUARTER, CURRENT_TIMESTAMP))
GROUP BY a.vertical
ORDER BY sum_proj_acv DESC

**Pipeline Coverage by Quarter**

_All pipeline coverage by close quarter_

In [ ]:
/* Pipeline Coverage by Quarter */
SELECT 
    YEAR(close_date) AS close_year,
    DATEPART(QUARTER, close_date) AS close_quarter,
    COUNT(opportunity_id) AS num_opps, 
    SUM(projected_acv) AS sum_proj_acv, 
    AVG(projected_acv) AS avg_proj_acv
FROM Opportunities
WHERE 
    opportunity_type LIKE 'New Booking%' 
    AND customer_type = 'Direct' 
    AND stage <> 'Closed - Won' 
    AND stage <> 'Closed - Lost'
GROUP BY DATEPART(QUARTER, close_date), YEAR(close_date)

## **Marketing Qualified Leads (MQLs)**

_Shows the number of MQLs split up in different views_

**Total MQLs**

_Shows the number of MQLs grouped by week number and shows the week dates for a specific week number_

In [ ]:
/* Total Number of MQLs */
SELECT COUNT(*) AS total_mqls
FROM Accounts
WHERE most_recent_mql_date IS NOT NULL

/* MQLs by Week Number */
SELECT 
    DATEPART(WEEK, most_recent_mql_date) AS week_num,
    COUNT(*) AS total_mqls
FROM Accounts
WHERE most_recent_mql_date IS NOT NULL
GROUP BY DATEPART(WEEK, most_recent_mql_date)
ORDER BY week_num

**Total MQLs by Week**

_Show the number of MQLs grouped by the week number and show the start and end date for the week (note: shows only the range of dates in which there was an MQL)_

In [ ]:
/* Create a CTE with a list of week numbers and their start and end dates */
WITH WeekData AS (
    SELECT DISTINCT DATEPART(WEEK, most_recent_mql_date) AS WeekNumber,
           MIN(most_recent_mql_date) AS StartDate,
           MAX(most_recent_mql_date) AS EndDate
    FROM Accounts
    GROUP BY DATEPART(WEEK, most_recent_mql_date)
)

/* Join the WeekData CTE with Accounts table and calculate the count */
SELECT
    WeekData.WeekNumber,
    WeekData.StartDate,
    WeekData.EndDate,
    COUNT(most_recent_mql_date) AS MQLCount
FROM WeekData
JOIN Accounts ON DATEPART(WEEK, Accounts.most_recent_mql_date) = WeekData.WeekNumber
WHERE most_recent_mql_date IS NOT NULL
GROUP BY WeekData.WeekNumber, WeekData.StartDate, WeekData.EndDate
ORDER BY StartDate, WeekData.WeekNumber;

**MQLs by Year & Quarter**

_Shows the number of MQLs for each quarter in each year_

In [ ]:
/* MQLs by Year and Quarter */
SELECT
    DATEPART(YEAR, most_recent_mql_date) AS year,
    DATEPART(QUARTER, most_recent_mql_date) AS quarter,
    COUNT(most_recent_mql_date)
FROM Accounts
GROUP BY 
    DATEPART(YEAR, most_recent_mql_date), 
    DATEPART(QUARTER, most_recent_mql_date)
ORDER BY year, quarter

**MQLs QTD by Account Rank**

In [ ]:
/* MQLs QTD by Account Rank */
SELECT
    account_rank, 
    COUNT(most_recent_mql_date)
FROM Accounts
WHERE DATEPART(QUARTER, most_recent_mql_date) = DATEPART(QUARTER, CURRENT_TIMESTAMP)
GROUP BY account_rank

**\# of MQLs for Specified Week & Avg MQLs Prior 4 Weeks**

_Shows the # of MQLs for specified week and then compares to the average # of MQLs from the past 4 weeks_

In [ ]:
/* Put in the date you would like to check */
DECLARE @GivenDate DATE = '06-23-2023';

/* Calculate week start (Sunday) and end (Saturday) dates for the given date */
DECLARE @WeekStartDate DATE = DATEADD(DAY, 1 - DATEPART(WEEKDAY, @GivenDate), @GivenDate);
DECLARE @WeekEndDate DATE = DATEADD(DAY, 7 - DATEPART(WEEKDAY, @GivenDate), @GivenDate);

/* Show the calculated week start and end dates */
SELECT 
    @GivenDate AS GivenDate, 
    @WeekStartDate AS WeekStartDate, 
    @WeekEndDate AS WeekEndDate,
    COUNT(most_recent_mql_date) AS mqls_specified_week
FROM Accounts
WHERE most_recent_mql_date >= @WeekStartDate AND most_recent_mql_date <= @WeekEndDate

/* Show the Avg number of MQLs for 4 weeks prior to the specified week */
SELECT
    AVG(MQLCount) AS AvgMQLsPerWeek
FROM (
    SELECT
        DATEPART(WEEK, most_recent_mql_date) AS WeekNumber,
        SUM(CASE WHEN most_recent_mql_date IS NOT NULL THEN 1 ELSE 0 END) AS MQLCount
    FROM
        Accounts
    WHERE
        DATEDIFF(WEEK, most_recent_mql_date, @GivenDate) BETWEEN 1 AND 4
    GROUP BY
        DATEPART(WEEK, most_recent_mql_date)
) AS WeeklyMQLs;

## **Events**

**Total Discos and Total Demos**

_Getting a sense of the total discos and demos split up in various ways_

In [ ]:
/* How many total discos and total demos have been scheduled? */
SELECT
    SUM(CASE WHEN event_type = 'Meeting - Disco' THEN 1 ELSE 0 END) AS NumberOfDiscos,
    SUM(CASE WHEN event_type = 'Meeting - Demo' THEN 1 ELSE 0 END) AS NumberOfDemos
FROM
    Events;

/* How many total discos and total demos have been completed? */
SELECT
    SUM(CASE WHEN event_type = 'Meeting - Disco' THEN 1 ELSE 0 END) AS NumberOfDiscos,
    SUM(CASE WHEN event_type = 'Meeting - Demo' THEN 1 ELSE 0 END) AS NumberOfDemos
FROM
    Events
WHERE event_status = 'Complete';

**Total Discos by Year & Quarter**

In [ ]:
/* How many completed discos by year and quarter? */
SELECT
    DATEPART(YEAR, completed_date) AS year,
    DATEPART(QUARTER, completed_date) AS quarter,
    COUNT(completed_date) DiscosCompleted
FROM Events
WHERE event_type = 'Meeting - Disco'
GROUP BY 
    DATEPART(YEAR, completed_date), 
    DATEPART(QUARTER, completed_date)
ORDER BY year, quarter

**Discos QTD by Expected Service Tier**

In [ ]:
/* Discos QTD by EST */
SELECT
    a.expected_service_tier,
    COUNT(e.completed_date) AS DiscosCompleted
FROM Events e LEFT JOIN Accounts a ON e.account_id = a.account_id
WHERE 
    e.event_type = 'Meeting - Disco'
    AND DATEPART(QUARTER, e.completed_date) = DATEPART(QUARTER, CURRENT_TIMESTAMP)
    AND DATEPART(YEAR, e.completed_date) = DATEPART(YEAR, e.completed_date)
GROUP BY a.expected_service_tier
ORDER BY a.expected_service_tier

**Accounts with 2+ Discos In Current Year**

In [ ]:
/* Accounts with 2+ Discos in CY */
SELECT a.account_name, a.expected_service_tier, COUNT(e.event_id) NumEvents
FROM Accounts a INNER JOIN Events e ON a.account_id = e.account_id
WHERE
    e.event_type = 'Meeting - Disco' 
    AND e.event_status = 'Complete'
    AND DATEPART(YEAR, e.completed_date) = DATEPART(YEAR, CURRENT_TIMESTAMP)
GROUP BY a.account_name, a.expected_service_tier
HAVING COUNT(e.event_id) >= 2

**Accounts with Events and Opportunities Created After Event**

In [ ]:
/* Accounts with events that had an opportunity created after the event was complete */
SELECT DISTINCT
    a.account_id,
    a.account_name,
    o.opportunity_name
    o.projected_acv,
    o.stage
FROM
    Accounts a
INNER JOIN
    Events e ON a.account_id = e.account_id
INNER JOIN
    Opportunities o ON a.account_id = o.account_id
WHERE
    event_status = 'Complete'
    AND o.created_date >= e.completed_date;